[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Regressor - Polyfit with Lasso Regularization

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 01/10/2022 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/IntroductionMachineLearningSystemEngineers/RegressorLasso.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.linear_model import lars_path, Lasso, lasso_path
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures

from scipy.spatial.distance import cdist

# Misc
import datetime
import os
from platform import python_version
import random
import warnings
import yaml

# Typing
from typing import Tuple

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

In [ ]:
# Configuration
%matplotlib inline

warnings.filterwarnings("ignore")

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


In [ ]:
# Parameters

# Data Generation
numSamples  = 50
noiseStd    = 0.3

# Model
vP = np.array([0.25, 2, 5])
polynomDeg = 2
λ = 0.1

# Data Visuzalization
gridNoiseStd = 0.05
numGridPts = 250

In [ ]:
# Auxiliary Functions

def PlotPolyFitLasso( vX: np.ndarray, vY: np.ndarray, vP: np.ndarray = None, P: int = 1, λ: float = 0.0, numGridPts: int = 1001, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, markerSize: int = MARKER_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, axisTitle: str = None ):

    if hA is None:
        hF, hA = plt.subplots(1, 2, figsize = figSize)
    else:
        hF = hA[0].get_figure()

    numSamples = len(vY)

    # Polyfit
    if λ == 0:
        # No Lasso (Classic Polyfit)
        vW  = np.polyfit(vX, vY, P)
    else:
        # Lasso
        mX   = PolynomialFeatures(degree = P, include_bias = False).fit_transform(vX[:, None])
        oMdl = Lasso(alpha = λ, fit_intercept = True, max_iter = 30000).fit(mX, vY)
        # Lasso coefficients
        vW   = np.r_[oMdl.coef_[::-1], oMdl.intercept_]
    
    # R2 Score
    vHatY = np.polyval(vW, vX)
    R2    = r2_score(vY, vHatY)
    
    # Plot
    xx  = np.linspace(np.around(np.min(vX), decimals = 1) - 0.1, np.around(np.max(vX), decimals = 1) + 0.1, numGridPts)
    yy  = np.polyval(vW, xx)

    hA[0].plot(vX, vY, '.r', ms = 10, label = '$y_i$')
    hA[0].plot(xx, yy, 'b',  lw = 2,  label = '$\hat{f}(x)$')
    hA[0].set_title (f'P = {P}, R2 = {R2}')
    hA[0].set_xlabel('$x$')
    hA[0].set_xlim(left = xx[0], right = xx[-1])
    hA[0].set_ylim(bottom = np.floor(np.min(vY)), top = np.ceil(np.max(vY)))
    hA[0].grid()
    hA[0].legend()
    
    hA[1].stem(vW[::-1], label = 'Estimated')
    if vP is not None:
        hA[1].stem(vP[::-1], linefmt = None, markerfmt = 'D', label = 'Ground Truth')
    hA[1].set_title('Coefficients')
    hA[1].set_xlabel('$w$')
    hA[1].set_ylim(bottom = -2, top = 6)
    hA[1].legend()

    # return hF



## Generate Data

$$ y_{i} = f \left( x_{i} \right) + \epsilon_{i} $$

Where

$$ f \left( x \right) = \frac{1}{2} x^{2} + 2x + 5 $$

In [ ]:
def f( vX ):
    # return 0.25 * (vX ** 2) + 2 * vX + 5
    return np.polyval(vP, vX)

vX = np.linspace(-2, 2, numSamples, endpoint = True) + (gridNoiseStd * np.random.randn(numSamples))
vN = noiseStd * np.random.randn(numSamples)
vY = f(vX) + vN


### Plot Data

In [ ]:
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA.plot(vX, vY, '.r', ms = MARKER_SIZE_DEF, label = r'$y_i = \frac{2}{3}x_i^2 + 2x_i + 5 + \epsilon_i$')
hA.set_xlabel('$x$')
hA.legend()
hA.grid()

## Train Polyfit Regressor with Lasso Regularization

$$\arg\min_{\boldsymbol{w},b}\left\Vert \boldsymbol{y}-\boldsymbol{\Phi}\boldsymbol{w} - b \boldsymbol{1} \right\Vert _{2}^{2}+\lambda\left\Vert \boldsymbol{w}\right\Vert _{1}$$

Where

$$\boldsymbol{\Phi}=\left[\begin{matrix}x_{1} & x_{1}^{2} & \cdots & x_{1}^{P}\\
x_{2} & x_{2}^{2} & \cdots & x_{2}^{P}\\
\vdots & \vdots &  & \vdots\\
x_{N} & x_{N}^{2} & \cdots & x_{N}^{P}
\end{matrix}\right]$$

In [ ]:
# Polynomial Fit with Lasso Regularization

mX   = PolynomialFeatures(degree = polynomDeg, include_bias = False).fit_transform(vX[:, None]) #<! Build the model matrix
oMdl = Lasso(alpha = λ, fit_intercept = True, max_iter = 30000).fit(mX, vY)
vW   = np.r_[oMdl.coef_[::-1], oMdl.intercept_]
vW

### Plot Regressor for Various Regularization (λ) Values

In [ ]:
hPolyFitLasso = lambda λ: PlotPolyFitLasso(vX, vY, vP = vP, P = 15, λ = λ)
lamSlider = FloatSlider(min = 0, max = 2, step = 0.0001, value = 0, readout_format = '.4f', layout = Layout(width = '30%'))
interact(hPolyFitLasso, λ = lamSlider)
plt.show()

## Lasso Path for Feature Importance

The _rise_ of a feature is almost equivalent to the correlation.  
Hence we cen use the _Lasso Path_ for feature selcection / significance.

In [ ]:
# Data from https://gist.github.com/seankross/a412dfbd88b3db70b74b
# mpg - Miles per Gallon
# cyl - # of cylinders
# disp - displacement, in cubic inches
# hp - horsepower
# drat - driveshaft ratio
# wt - weight
# qsec - 1/4 mile time; a measure of acceleration
# vs - 'V' or straight - engine shape
# am - transmission; auto or manual
# gear - # of gears
# carb - # of carburetors
dfMpg = pd.read_csv('https://github.com/FixelAlgorithmsTeam/FixelCourses/raw/master/DataSets/mtcars.csv')
dfMpg

In [ ]:
# Data for Analysis
dfX = dfMpg.drop(columns = ['model', 'mpg'], inplace = False)
dfX = (dfX - dfX.mean()) / dfX.std() #<! Normalize
dsY = dfMpg['mpg'].copy() #<! Data Series

In [ ]:
# LASSO Path Analysis

alphasPath, coefsPath, *_ = lasso_path(dfX, dsY)
# alphasPath, coefsPath, *_ = lars_path(dfX, dsY, method = 'lasso')




In [ ]:
hF, hA = plt.subplots(figsize = (24, 16))
hA.plot(alphasPath, np.abs(coefsPath.T), lw = 2, label = dfX.columns)
hA.set_title('The Lasso Path')
hA.set_xlabel('$\lambda$')
hA.set_ylabel('Coefficient Value')
hA.legend()
hF.show()
